In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('../Downloads/archive'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [3]:
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm.notebook import tqdm
from glob import glob
from keras.layers import Flatten, Dense
from keras.applications.vgg16 import VGG16
import xml.etree.ElementTree as xet
from keras.models import load_model, Sequential
from sklearn.model_selection import train_test_split

In [10]:
def load_images(directory):
    images = []

    for filepath in tqdm(os.listdir(directory)):
        img_path = os.path.join(directory,filepath)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(224,224))
        img = cv2.medianBlur(img,5)
        images.append(img)
    
    images = np.array(images).reshape(len(images),224,224,3)
    return images

In [ ]:
%ls

In [ ]:
path = glob("../archive/annotations/*.xml")
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])

In [ ]:
for filename in path:
    data = xet.parse(filename)
    root = data.getroot()
    obj = root.find('object')
    labels_info = obj.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)
    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)


In [ ]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

In [ ]:
df.shape


In [ ]:
images = load_images('../archive/images/')

In [ ]:
images.shape


In [ ]:
filename = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('../archive/images',filename_image)
    return filepath_image
getFilename(filename)

In [ ]:
image_path = list(df['filepath'].apply(getFilename))
image_path[:10]

In [ ]:
bbox_coords = df.iloc[:,1:].values
labels = []

for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    xmin,xmax,ymin,ymax = bbox_coords[ind]
    nxmin, nxmax = int(xmin/(w/255.)), int(xmax/(w/255.))
    nymin, nymax = int(ymin/(h/255.)), int(ymax/(h/255.))
    labels.append((nxmin,nxmax,nymin,nymax))

In [ ]:
labels = np.array(labels)
labels

In [ ]:
X = np.array(images,dtype=np.float32)
y = np.array(labels,dtype=np.float32)

In [ ]:
X = X / 255.
y = y / 255.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.1,random_state=42)

In [ ]:
model = Sequential()
model.add(VGG16(weights="imagenet", input_shape=(224, 224, 3)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(4, activation="sigmoid"))

model.layers[-6].trainable = False

model.summary()

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


In [ ]:
r = model.fit(X_train,
              y_train,
              epochs=50,
              batch_size=32,
              validation_data=(X_val,y_val),
              verbose=1)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(r.history['loss'],'r',label='train loss')
plt.plot(r.history['val_loss'],'b',label='validation loss')
plt.xlabel('No. of Epochs')
plt.ylabel('Loss')
plt.title('Loss Graph')
plt.legend();

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(r.history['accuracy'],'r',label='train accuracy')
plt.plot(r.history['val_accuracy'],'b',label='validation accuracy')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Graph')
plt.legend();

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(r.history['accuracy'],'r',label='train accuracy')
plt.plot(r.history['val_accuracy'],'b',label='validation accuracy')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy Graph')
plt.legend();

In [ ]:
loss, acc = model.evaluate(X_test,y_test)
print("Test Loss:",loss)
print("Test Accuracy:",acc)

In [13]:
detector = load_model('license_plate_detector.h5')
detector

2023-03-25 13:43:07.165900: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-25 13:43:07.167057: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [16]:
l = load_images('/Users/paramkansagra/Downloads/archive/test')
X = np.array(l,dtype=np.float32)
arr = np.int16(detector.predict(X)[0]*255)
x , y , w , h = arr[0],arr[1],arr[2],arr[3]

image = cv2.imread("/Users/paramkansagra/Downloads/archive/test/Cars421.png",0)

startPoint = (x,y)
endPoint = (x+h,w+y)

print("hello")
print(startPoint,endPoint)

color = (255,0,0)
thickness = 2

res = cv2.rectangle(image,startPoint,endPoint,color,thickness)
cv2.imshow("idk",res)
cv2.waitKey(0)
cv2.destroyAllWindows()

  0%|          | 0/2 [00:00<?, ?it/s]

None


error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
